In [1]:
# !pip install pennylane

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [3]:
n_qubits = 3
depth = 4

In [4]:
dev = qml.device("default.qubit",wires=n_qubits)
dev

<default.qubit device (wires=3) at 0x7ebb38000880>

In [5]:
def layer(layer_weights):
    for dep in range(depth):
      #L1
      for wire in range(n_qubits):
        qml.RY(layer_weights[0][wire+(n_qubits*dep)], wires=wire)

      qml.CZ(wires=[0,1])
      qml.CZ(wires=[1,2])
      qml.CZ(wires=[2,0])

In [6]:
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

In [7]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [8]:
@qml.qnode(dev,diff_method="backprop")
def circuit(weights):
  layer(weights)
  return qml.state()

In [9]:
def variational_classifier(weights, x):
    return np.real(circuit(weights))

In [10]:
x = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])
weights = np.random.rand(1,12)
print(weights)
predictions_test = variational_classifier(weights, x)
print(predictions_test)
print(accuracy(x,predictions_test))

[[0.83564626 0.44553789 0.42664439 0.29032953 0.59814099 0.15332369
  0.61051376 0.07471093 0.77787382 0.80956403 0.85563999 0.8112834 ]]
[-0.28537337  0.42878701  0.34836967 -0.28137478  0.57407977 -0.1391902
 -0.42926743 -0.03096001]
0.005263268686518836


In [11]:
def F1_loss(a,Appro_a):
  #   Define the cost function
  F1=(a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2));
  F2=(a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2));
  F3=(a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2));

  #   Prepare the Bell state
  Bell_State=np.ones((8,1))/np.linalg.norm(np.ones((8,1)));
  #   The sum of vector a
  Sum_a=np.sum(a);

  sqrt_D = np.linalg.norm(np.ones((8,1)))
  UaFunction=np.abs(Sum_a - (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F1+F2+F3)

  return np.real(UaFunction)

In [12]:
def cost(weights, X):
    predictions = variational_classifier(weights, X)
    return F1_loss(X, predictions)

In [13]:
# opt = NesterovMomentumOptimizer(0.5)
opt = qml.AdamOptimizer()
np.random.seed(0)

In [14]:
# delta=1;
# Omega=1;
# gamma=1;
# Delta_t=0.01;

# alpha=np.sqrt(2)/2*delta*Delta_t;
# beta=(1-gamma*Delta_t/2);
# aa=[Delta_t*np.sqrt(2)/2*delta,Delta_t*gamma/4,np.sqrt(alpha**2+beta**2),Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4]
# x=np.sqrt(aa);
# norma=np.linalg.norm(x);
# x=x/np.linalg.norm(x);

# print(x)

In [15]:
x = np.array([0.08337992,0.049578,0.98909045,0.049578,0.049578,0.049578,0.049578,0.049578])

sum = 0
for i in range(len(x)):
  sum += (x[i] ** 2)

print(sum)

0.9999999978444085


In [16]:
weights = 2*np.pi*np.random.rand(1,12)
print(weights)

for it in range(1000):
    # Update the weights by one optimizer step, using only a limited batch of data
    weights = opt.step(cost, weights, X=x)
    # Compute accuracy
    predictions = variational_classifier(weights, x)

    acc = accuracy(x, predictions)

    print(f"Iter: {it+1:4d} | Accuracy: {acc:0.7f}")
    print(weights)
    print("-----------------------------------------------------------")

[[3.44829694 4.49366732 3.78727399 3.42360201 2.66190161 4.0582724
  2.74944154 5.60317502 6.0548717  2.40923412 4.97455513 3.32314479]]
Iter:    1 | Accuracy: 0.0429835
[[3.43829695 4.48366732 3.79727398 3.43360201 2.67190161 4.0682724
  2.75944153 5.59317502 6.0648717  2.41923412 4.96455513 3.31314479]]
-----------------------------------------------------------
Iter:    2 | Accuracy: 0.0494294
[[3.42831579 4.47367823 3.80722163 3.44361085 2.68187901 4.07824966
  2.76920369 5.58319465 6.07485293 2.42922941 4.95456935 3.30316544]]
-----------------------------------------------------------
Iter:    3 | Accuracy: 0.0563634
[[3.41836564 4.46370699 3.81716837 3.45363294 2.69181877 4.08818961
  2.77843375 5.57324645 6.08480325 2.43921704 4.94460771 3.29321966]]
-----------------------------------------------------------
Iter:    4 | Accuracy: 0.0637536
[[3.40845833 4.45376023 3.82714563 3.4636714  2.70170591 4.0980781
  2.78668943 5.56334264 6.09471094 2.44919414 4.9346803  3.28331974]]
-

In [19]:
# Iter: 1000 | Accuracy: 0.4140536

weights = np.array([[2.90243385,3.96707059,3.68918657,3.91572417,2.7618946,4.39657892,2.48758716,5.53744781,6.64766312,2.9903162,4.37867459,2.71508533]])


predictions_test = variational_classifier(weights, x)
print(predictions_test)
print(x)
print(accuracy(x,predictions_test))

[ 0.28384004  0.27431034 -0.74173339  0.27142494  0.27005507  0.27238777
  0.27007501 -0.0168342 ]
[0.08337992 0.049578   0.98909045 0.049578   0.049578   0.049578
 0.049578   0.049578  ]
0.41405363961524505


In [18]:
state0 = qml.math.dm_from_state_vector([0.4472135954999579,0,0.7071067811865476,0,0,0,0.4472135954999579,0.31622776601683794])
state1 = qml.math.dm_from_state_vector([0.4472135954999579,0,0.7071067811865476,0,0,0,0.4472135954999579,0.31622776601683794])
qml.math.fidelity(state0, state1)

1.0000000214620772